In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    AutoTokenizer,
    set_seed,
)
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import torch
from transformers import BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model


In [2]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
def preprocess_function(examples, **fn_kwargs):
    return fn_kwargs["tokenizer"](examples["text"], truncation=True)


def get_data(train_path, test_path, random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)

    train_df, val_df = train_test_split(
        train_df, test_size=0.2, stratify=train_df["label"], random_state=random_seed
    )

    return train_df, val_df, test_df


def compute_metrics(eval_pred):
    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    results = {}
    results.update(
        f1_metric.compute(predictions=predictions, references=labels, average="micro")
    )

    return results

In [3]:
print_gpu_utilization()

GPU memory occupied: 593 MB.


In [4]:
random_seed = 0
train_path = "SubtaskA/subtaskA_train_monolingual.jsonl"
test_path = "SubtaskA/subtaskA_dev_monolingual.jsonl"
model_id = "NousResearch/Llama-2-7b-hf"  # For example 'xlm-roberta-base'
subtask = "SubtaskA"
experiment_type = "mono"
prediction_path = "out.json"

In [5]:
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1}

In [6]:
set_seed(random_seed)

In [7]:
train_df, valid_df, test_df = get_data(train_path, test_path, random_seed)

In [8]:
checkpoints_path = f"{experiment_type}_{model_id}/subtask{subtask}/{random_seed}"

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at NousResearch/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.config.use_cache = False

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id, model_max_length=2048)

In [12]:
from peft import prepare_model_for_int8_training
model = prepare_model_for_kbit_training(model)

In [13]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="SEQ_CLS",
)

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 33570816 || all params: 3402911744 || trainable%: 0.9865320797458801


In [16]:
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

In [18]:
# tokenize data for train/valid
tokenized_train_dataset = train_dataset.map(
    preprocess_function, batched=True, fn_kwargs={"tokenizer": tokenizer}
)
tokenized_valid_dataset = valid_dataset.map(
    preprocess_function, batched=True, fn_kwargs={"tokenizer": tokenizer}
)


Map:   0%|          | 0/95805 [00:00<?, ? examples/s]

Map:   0%|          | 0/23952 [00:00<?, ? examples/s]

In [19]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# create Trainer
training_args = TrainingArguments(
    output_dir=checkpoints_path,
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    tf32=True,
    disable_tqdm=False,
    #gradient_accumulation_steps=1,
    gradient_checkpointing=False,
)

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(adam_bnb_optim, None)
)

trainer.train()

# save best model
best_model_path = checkpoints_path + "/best/"

if not os.path.exists(best_model_path):
    os.makedirs(best_model_path)

trainer.save_model(best_model_path)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
